# Star Wars Rebels (DVD)

<https://thetvdb.com/series/star-wars-rebels>

![Star Wars Rebels Clearart](https://artworks.thetvdb.com/banners/v4/series/283468/clearart/611b7d0eb3bea.png)


These notes preserve my memories of auto-generating [Kodi](https://kodi.tv/) [[GitHub](https://github.com/xbmc)] `tvshow` and `episodedetails` XML files in `*.nfo` formats.

This `*.nfo` format is kind of covered in the following documents:

- [NFO files](https://kodi.wiki/view/NFO_files)
- [NFO files/TV shows](https://kodi.wiki/view/NFO_files/TV_shows)
- [NFO files/Episodes](https://kodi.wiki/view/NFO_files/Episodes)


In [1]:
%run beautifulsoup-thetvdb.py

from IPython.core.display import display, HTML


## generating `tvshow` data


In [2]:
series_location = 'https://thetvdb.com/series/star-wars-rebels'
series_soup = getSoup(series_location)

series_basic_info = dict(yieldSeriesBasicInfo(series_soup))

series_data = dict(
    uniqueids=[
        {
            'type': 'tvdb',
            'uniqueid': 'series/star-wars-rebels'
        },
        {
            'type': 'imdb',
            'uniqueid': 'tt2930604'
        },
    ],
    title=getSeriesTitle(series_soup),
    plot=getSeriesPlot(series_soup),
    thumbs=[
        {
            'aspect': 'banner',
            'dim': None,
            'season': None,
            'src': 'https://artworks.thetvdb.com/banners/graphical/283468-g4_t.jpg'
        },
        {
            'aspect': 'poster',
            'dim': None,
            'season': None, # maps to the top-level movie poster
            'src': 'https://artworks.thetvdb.com/banners/posters/283468-5_t.jpg'
        },
        {
            'aspect': 'poster',
            'dim': None,
            'season': 1,
            'src': 'https://artworks.thetvdb.com/banners/posters/283468-5_t.jpg'
        },
        {
            'aspect': 'poster',
            'dim': None,
            'season': 2,
            'src': 'https://artworks.thetvdb.com/banners/posters/283468-6.jpg'
        },
        {
            'aspect': 'fanart',
            'dim': '640x360',
            'season': 1,
            'src': 'https://artworks.thetvdb.com/banners/fanart/original/283468-2_t.jpg'
        },
        {
            'aspect': 'fanart',
            'dim': '640x360',
            'season': 2,
            'src': 'https://artworks.thetvdb.com/banners/fanart/original/283468-13_t.jpg'
        },
    ],
    genres=series_basic_info['Genres'],
    actors=list(yieldTVShowActors(series_soup))
)

getTVShowXmlTree(series_data).write(
    './beautifulsoup-star-wars-rebels/tvshow.nfo',
    encoding='utf-8',
    xml_declaration=True)

## generating `episodedetails` data


In [3]:

# dvd_episode_location = 'https://thetvdb.com/series/star-wars-rebels/seasons/dvd/1'

dvd_episode_location = 'https://thetvdb.com/series/star-wars-rebels/seasons/dvd/2'

dvd_episode_soup = getSoup(dvd_episode_location)
table = dvd_episode_soup.find('table')

display(HTML(table.decode()))

,Name,First Aired,Runtime,Image,Certified
S02E01,The Siege of Lothal (1),"June 20, 2015 Disney XD",25,,
S02E02,The Siege of Lothal (2),"June 20, 2015 Disney XD",25,,
S02E03,The Lost Commanders,"October 14, 2015 Disney XD",25,,
S02E04,Relics of the Old Republic,"October 21, 2015 Disney XD",25,,
S02E05,Always Two There Are,"October 28, 2015 Disney XD",25,,
S02E06,Brothers of the Broken Horn,"November 4, 2015 Disney XD",25,,
S02E07,Wings of the Master,"November 11, 2015 Disney XD",25,,
S02E08,Blood Sisters,"November 18, 2015 Disney XD",25,,
S02E09,Stealth Strike,"November 25, 2015 Disney XD",25,,
S02E10,The Future of the Force,"December 2, 2015 Disney XD",25,,


In [4]:
episode_data = list(yieldEpisodeData(table))
episode_data

[{'episode': 'S02E01',
  'title': 'The Siege of Lothal (1)',
  'plotUri': '/series/star-wars-rebels/episodes/7460328',
  'plot': 'Now a part of Phoenix Squadron, the Ghost crew joins a mission to steal shield generators from an Imperial freighter.',
  'year': '2015'},
 {'episode': 'S02E02',
  'title': 'The Siege of Lothal (2)',
  'plotUri': '/series/star-wars-rebels/episodes/7460329',
  'plot': 'Under pressure from Darth Vader, an Imperial defector leads the Sith Lord to battle with Kanan and Ezra.',
  'year': '2015'},
 {'episode': 'S02E03',
  'title': 'The Lost Commanders',
  'plotUri': '/series/star-wars-rebels/episodes/5318870',
  'plot': 'Ahsoka sends the Rebel crew to find and recruit a war hero to their cause, but when they discover it is Captain Rex, trust issues put the mission at risk.',
  'year': '2015'},
 {'episode': 'S02E04',
  'title': 'Relics of the Old Republic',
  'plotUri': '/series/star-wars-rebels/episodes/5369302',
  'plot': "Ezra and the others are starting to form

In [5]:
nfo_location_template = f"./beautifulsoup-star-wars-rebels/{series_data['title']}"

In [6]:
for item in episode_data:
    episode = item['episode']
    year = item['year']
    xml_tree = getEpisodeDetailsXmlTree(item['title'], item['plot'])
    writeEpisodeDetailsXml(nfo_location_template, episode, year, xml_tree)
